In [3]:
from redisvl.extensions.llmcache import SemanticCache

redis_url = "redis://localhost:6379"

cache = SemanticCache(
    name="test_optimize_threshold_cache",
    redis_url=redis_url,
)

paris_key = cache.store(prompt="what is the capital of france?", response="paris")
rabat_key = cache.store(prompt="what is the capital of morocco?", response="rabat")

test_data = [
    {"query": "what actually is the capital of france?", "query_match": paris_key},
    {"query": "what actually is the capital of morocco?", "query_match": rabat_key},
    {"query": "What is the state bird of virginia?", "query_match": ""},
]

10:47:23 redisvl.index.index INFO   Index already exists, not overwriting.


In [14]:
from redisvl.query.query import BaseQuery
_query = BaseQuery().return_field("prompt_vector", decode_field=False)
cached_records = cache.index.query(_query)

In [15]:
test_data_embeddings = cache._vectorizer.embed_many([test.query_match for test in test_data])

[{'id': 'test_optimize_threshold_cache:c990cc06e5e77570e5f03360426d2b7f947cbb5a67daa8af8164bfe0b3e24fe3',
  'prompt_vector': b'\xccH\xb1<2\xe0\xb0<\xa0\x81\x0c;\xfe\xb5a=\xa0\x9a1\xbd!\xc4\xf8\xbbm/(\xbdV\x93\n\xbc\x18.\x88<\x98{\n\xbc\x0er\x10=P\x03G\xbd\xe4\xa4\x9e<\xb9\xad\x05\xbdu\xae\x85=#w7\xbb\xdak\xf3;C\xf5&\xbd\x15+y\xbc\xcf\xa3c\xbb4\xb8\xb4<\xbb\x98\x08=\xa2p\x10<qN\xae<D\x94\xdc<\xb3\xdc\x9c=\xc8\xe8Z\xbd\xa7}$<h!\xf6\xbb\xe1\xc4\xb6=\xb0\xd8\xf4<\x96\xaa\x84\xbc\x1d\xde\x1e<\xd8\xff\x8a\xbc\x14D\xc35\x9d\xb4\x87<\xcd \xe7\xbc\x8c\xf3\x8e<\xf8\xcfa<Y\x97\x0f\xbdA))\xbd\xbe\xd2\x9f:^\x88\xc5\xbav|\x00=/c\t=l]\xa3\xbc\x8e\x8a\x12=3(\xc6<\x8f\xebX\xbd\x16=!;@\xf7n<\x18\xf6\x0e\xbc\x80\xfa\x06\xbdo \xe1\xbc\xcc=S\xbd!\x86\xca<\xc2\xb3s\xbcV\'z=9\xa0\xf0\xbc\x92\xef\x88;\x15\xa9\r=|\xc2\xee\xbcC\r\xec\xbc\x95\x9d\x93\xbc\xe1m!\xbd\xdeX\x02\xbd\xa4\rA=mNU=\x9fl!<\x08\xda\x12\xbara\x93\xbb\x92\x0e\x9b\xbc%d\x15<p\tr=s7\xbd\xbbq\x11\xed\xba2hn\xbci+\xc9\xbc71G=H\x16\x8c<\xa2A\x8e=}

In [3]:
cache.check("what's the capital of france?")

[{'entry_id': 'c990cc06e5e77570e5f03360426d2b7f947cbb5a67daa8af8164bfe0b3e24fe3',
  'prompt': 'what is the capital of france?',
  'response': 'paris',
  'vector_distance': 0.0847067832947,
  'inserted_at': 1740421206.85,
  'updated_at': 1740421206.85,
  'key': 'test_optimize_threshold_cache:c990cc06e5e77570e5f03360426d2b7f947cbb5a67daa8af8164bfe0b3e24fe3'}]

In [9]:
cache.check("what is the capital of morocco?")

[{'entry_id': 'e4a10696892cec3c263e5c3b661452867d3de4187fbfc87f92defb04e31b7f78',
  'prompt': 'what is the capital of morocco?',
  'response': 'rabat',
  'vector_distance': -3.57627868652e-07,
  'inserted_at': 1740421206.87,
  'updated_at': 1740421206.87,
  'key': 'test_optimize_threshold_cache:e4a10696892cec3c263e5c3b661452867d3de4187fbfc87f92defb04e31b7f78'}]

In [10]:
qrels_dict = {
    "q_1": {
        "test_optimize_threshold_cache:c990cc06e5e77570e5f03360426d2b7f947cbb5a67daa8af8164bfe0b3e24fe3": 1
    },
    "q_2": {
        "test_optimize_threshold_cache:e4a10696892cec3c263e5c3b661452867d3de4187fbfc87f92defb04e31b7f78": 1
    }
}

In [11]:
from ranx import Qrels
qrels = Qrels(qrels_dict, name="test")

In [12]:
from ranx import Run

run_dict = {
    "q_1": {
        "test_optimize_threshold_cache:c990cc06e5e77570e5f03360426d2b7f947cbb5a67daa8af8164bfe0b3e24fe3": 1
    },
    "q_2": {}
}

run = Run(run_dict, name="test")

In [13]:
from ranx import evaluate
evaluate(qrels, run, "f1")

0.5

In [6]:
from redisvl.query import RangeQuery

queries = [
    {
        "q_id": "q_1",
        "query": "what's the capital of france??"
    },
    {
        "q_id": "q_2",
        "query": "what's the capital of morocco??"
    },
]

for q in queries:
    vec = cache._vectorizer.embed(q["query"])

    query = RangeQuery(
        vec,
        vector_field_name="prompt_vector",
        distance_threshold=1.0
    )

    res = cache.index.query(query)

    q["response"] = res

In [7]:
queries

[{'q_id': 'q_1',
  'query': "what's the capital of france??",
  'response': [{'id': 'test_optimize_threshold_cache:c990cc06e5e77570e5f03360426d2b7f947cbb5a67daa8af8164bfe0b3e24fe3',
    'vector_distance': '0.122092962265'},
   {'id': 'test_optimize_threshold_cache:e4a10696892cec3c263e5c3b661452867d3de4187fbfc87f92defb04e31b7f78',
    'vector_distance': '0.412750303745'}]},
 {'q_id': 'q_2',
  'query': "what's the capital of morocco??",
  'response': [{'id': 'test_optimize_threshold_cache:e4a10696892cec3c263e5c3b661452867d3de4187fbfc87f92defb04e31b7f78',
    'vector_distance': '0.0875742435455'},
   {'id': 'test_optimize_threshold_cache:c990cc06e5e77570e5f03360426d2b7f947cbb5a67daa8af8164bfe0b3e24fe3',
    'vector_distance': '0.398189604282'}]}]

In [1]:
from ranx import evaluate, Qrels, Run

In [9]:
def generate_run(queries, threshold):
    run_dict = {}
    for query in queries:
        run_dict[query["q_id"]] = {}
        for res in query["response"]:
            if float(res["vector_distance"]) < threshold:
                run_dict[query["q_id"]][res["id"]] = 1
    return run_dict


In [ ]:
thresholds = [0.1, 0.3, 0.5]

qrels_dict = {
    "q_1": {
        "test_optimize_threshold_cache:c990cc06e5e77570e5f03360426d2b7f947cbb5a67daa8af8164bfe0b3e24fe3": 1
    },
    "q_2": {
        "test_optimize_threshold_cache:e4a10696892cec3c263e5c3b661452867d3de4187fbfc87f92defb04e31b7f78": 1
    }
}

qrels = Qrels(qrels_dict)

metrics = {}
for threshold in thresholds:
    run_dict = generate_run(queries, threshold)
    run = Run(run_dict)
    f1 = evaluate(qrels, run, "f1")
    metrics[threshold] = {"F1": f1}

In [11]:
metrics

{0.1: 0.5, 0.3: 1.0, 0.5: 0.6666666666666666}

In [12]:
cache.check("Hey what's up")

[]

In [30]:
qrels_dict = {
    "01JN14CMQGRE2Z8989C33GCMKD": {
        "test_optimize_threshold_cache:c990cc06e5e77570e5f03360426d2b7f947cbb5a67daa8af8164bfe0b3e24fe3": 1
    },
    "01JN14CMQG4A0N0SC6NB3SWCB2": {
        "test_optimize_threshold_cache:e4a10696892cec3c263e5c3b661452867d3de4187fbfc87f92defb04e31b7f78": 1
    }
}

qrels = Qrels(qrels_dict)

run_dict = {
        '01JN14CMQGRE2Z8989C33GCMKD': {
            "none": 1
        }, 
        '01JN14CMQG4A0N0SC6NB3SWCB2': {
        }, 
}

run = Run(run_dict)

In [31]:
evaluate(qrels, run, "f1", make_comparable=True)

0.0

In [32]:
r = {}

if not r:
    print("hello")

hello
